In [ ]:
#Plain OLS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)

#varying the grid size:
for n in [200, 500, 1000]:    

    #maximum polynomial degree+1
    maxdegree = 9

    # Make data set: the longtitude values of the Northern Hemisphere in radiants.
    x = np.linspace(0,(np.pi)/2,n).reshape(-1, 1)

    #Northern H. parameters
    s0 = 1
    s2 = -0.473
    a0 = 0.675
    a2 = -0.192
    i2 = -0.165

    #flux function (eqn. (14) from Stone_1978)
    y = 0.5*(s0*a2+s2*a0+(2/7)*s2*a2-i2)*((np.sin(x))**3-np.sin(x))

    #noisy flux function
    y_noisy = np.random.normal(y, abs(y*0.05)) 

    #varying the test/training data percentage
    for test_size in [0.25, 0.5, 0.75]:
        print('number of datapoints:', n,', test data percentage:', 100*test_size,'%')
        
        #MSE OLS analysis
        MSE_train = np.zeros(maxdegree)
        MSE_test = np.zeros(maxdegree)

        #Correlation OLS analysis
        R2_train=np.zeros(maxdegree)
        R2_test=np.zeros(maxdegree)

        polydegree = np.zeros(maxdegree) 

        for degree in range(maxdegree):
            #polynomial fit
            polydegree[degree]=degree
            poly = PolynomialFeatures(degree=degree)
            X = poly.fit_transform(x)
            
            n_iter=100
            
            MSE_train_iter=np.zeros(n_iter)
            MSE_test_iter=np.zeros(n_iter)
            
            R2_train_iter=np.zeros(n_iter)
            R2_test_iter=np.zeros(n_iter)
            
            #we iterrate 100 times to avoid erratic behvious. At least while the noise is turned off...
            for iter in range (n_iter):

                #splitting of the data
                X_train, X_test, y_train, y_test = train_test_split(X, y_noisy, test_size=test_size, random_state=42) 
                
                #OLS regression, using matrix inversion
                OLSbeta = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train  
                #print(OLSbeta)
                ytilde = X_train @ OLSbeta 
                ypredict = X_test @ OLSbeta #<----I'm doing it here analytically, just to show what's inside the LinearRegression function. Doing a LinearRegression.predict at the test set yields the same results.
                #MSE prediction
                MSE_train_iter[iter]=MSE(y_train,ytilde) 
                MSE_test_iter[iter]=MSE(y_test,ypredict) 
                
                #Correlation prediction
                R2_train_iter[iter]=R2(y_train,ytilde) #train data, original vs predicted
                R2_test_iter[iter]=R2(y_test,ypredict) #test data original vs predicted

            #MSE prediction
            MSE_train[degree]=np.mean(MSE_train_iter) #train data, original vs predicted
            #print(MSE_train)
            MSE_test[degree]=np.mean(MSE_test_iter) #test data original vs predicted
            
            #Correlation prediction
            R2_train[degree]=np.mean(R2_train_iter) #train data, original vs predicted
            #print(R2_train)
            R2_test[degree]=np.mean(R2_test_iter) #test data original vs predicted
            
            if n==200 and test_size==0.25 and degree==6:
                print(OLSbeta)

        plt.plot(polydegree, MSE_train, label='MSE train')
        plt.plot(polydegree, MSE_test, label='MSE test')
        plt.xlabel('Pol. Degree')
        plt.ylabel('MSE')
        plt.legend()
        plt.show()
            
        plt.plot(polydegree, R2_test, label='R2 Test')
        plt.plot(polydegree, R2_train, label='R2 Train')
        plt.xlabel('Pol. Degree')
        plt.ylabel('R2')
        plt.legend()
        plt.show()

In [ ]:
#Bootstrap OLS

In [ ]:
#Bias Variance trade-off as a function of the model's complexity
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

MSE_Bootstraps_compare = np.zeros(maxdegree) #to compare with the C-V code later

#varying the grid size:
for n in [200, 500, 1000]:    

    #maximum polynomial degree+1
    maxdegree = 9
    
    for n_bootstraps in [1, 20, 100]:
        print('number of datapoints:', n,', number of bootstraps:', n_bootstraps)

        # Make data set.
        x = np.linspace(0,(np.pi)/2,n).reshape(-1, 1)

        #Northern H. parameters
        s0 = 1
        s2 = -0.473
        a0 = 0.675
        a2 = -0.192
        i2 = -0.165

        #flux function (eqn. (14) from Stone_1978)
        y = 0.5*(s0*a2+s2*a0+(2/7)*s2*a2-i2)*((np.sin(x))**3-np.sin(x))

        #noisy flux function
        y_noisy = np.random.normal(y, abs(y*0.05))

        MSE_Bootstrap = np.zeros(maxdegree)

        bias = np.zeros(maxdegree)
        variance = np.zeros(maxdegree)
        polydegree = np.zeros(maxdegree)

        #splitting of the data
        x_train, x_test, y_train, y_test = train_test_split(x, y_noisy, test_size=0.25, random_state=42) 
                                                                                        # the fact that I'm resplitting the data for each degree loop shouldn't and doesn't affect the final value of MSE 
                                                                                        #I want to keep the splitting out of the loop because of the resampling. It doens't seem to affect the results, though...
        for degree in range(maxdegree):

            #polynomial fit
            polydegree[degree] = degree
            model = make_pipeline(PolynomialFeatures(degree=degree), LinearRegression(fit_intercept=False)) #it's easier to use Python's Pol.fit and Lin. Regr. features for this part now. 
            ypredict = np.empty((y_test.shape[0], n_bootstraps))    

            for i in range(n_bootstraps):
                x_, y_ = resample(x_train, y_train)

                #applying Linear Regression to the training data set
                ypredict[:, i] = model.fit(x_, y_).predict(x_test).ravel()     

            MSE_Bootstrap[degree] = MSE(y_test, ypredict)
            
            if (n==200 and n_bootstraps == 20):
                MSE_Bootstraps_compare[degree]=MSE_Bootstrap[degree]#to compare with the C-V code later
                #print(MSE_Bootstraps_compare)
                
            bias[degree] = np.mean( (y_test - np.mean(ypredict, axis=1, keepdims=True))**2 )
            variance[degree] = np.mean( np.var(ypredict, axis=1, keepdims=True) )

        plt.plot(polydegree, MSE_Bootstrap, label='MSE')
        plt.plot(polydegree, bias, label='bias')
        plt.plot(polydegree, variance, label='Variance')
        plt.xlabel('Pol. Degree')
        plt.ylabel('Prediction Error')
        plt.legend()
        plt.show()
#print(MSE_Bootstraps_compare)

In [ ]:
#Cross validation OLS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

#grid size
n = 200 
#maximum polynomial degree+1
maxdegree = 9

# Make data set.
x = np.linspace(0,(np.pi)/2,n)#.reshape(-1, 1)

#Northern H. parameters
s0 = 1
s2 = -0.473
a0 = 0.675
a2 = -0.192
i2 = -0.165

#flux function (eqn. (14) from Stone_1978)
y = 0.5*(s0*a2+s2*a0+(2/7)*s2*a2-i2)*((np.sin(x))**3-np.sin(x))

#noisy flux function
y_noisy = np.random.normal(y, abs(y*0.05))

polydegree = np.zeros(maxdegree)
model = LinearRegression(fit_intercept=False)

# Number of folds
for k in [5, 10, n]: #n for the LOO-CV
    kfold = KFold(n_splits = k)

    # Perform the cross-validation to estimate MSE
    MSE_KFold = np.zeros((maxdegree, k))

    i = 0
    for degree in range(maxdegree):
        poly = PolynomialFeatures(degree=degree)
        polydegree[degree]=degree
        j = 0
        for train_inds, test_inds in kfold.split(x):
            xtrain = x[train_inds]
            ytrain = y_noisy[train_inds]

            xtest = x[test_inds]
            ytest = y_noisy[test_inds]

            Xtrain = poly.fit_transform(xtrain[:, np.newaxis])
            model.fit(Xtrain, ytrain[:, np.newaxis])

            Xtest = poly.fit_transform(xtest[:, np.newaxis])
            ypred = model.predict(Xtest)

            MSE_KFold[i,j] = MSE(ytest[:, np.newaxis], ypred)

            j += 1
        i += 1

    estimated_mse_KFold = np.mean(MSE_KFold, axis = 1)

    # Cross-validation using cross_val_score from sklearn along with KFold

    estimated_mse_sklearn = np.zeros(maxdegree)
    i = 0
    for degree in range(maxdegree):
        polydegree[degree]=degree
        poly = PolynomialFeatures(degree=degree)
        X = poly.fit_transform(x[:, np.newaxis])
        estimated_mse_folds = cross_val_score(model, X, y_noisy[:, np.newaxis], scoring='neg_mean_squared_error', cv=k)

        # cross_val_score return an array containing the estimated NEGATIVE mse for every fold.
        # we have to the the mean of every array in order to get an estimate of the mse of the model
        estimated_mse_sklearn[i] = np.mean(-estimated_mse_folds)

        i += 1

    ## Plot    
        
    if k==5:          
        plt.figure()
        plt.plot(polydegree, estimated_mse_sklearn, 'b', label = 'cross_val_score')
        plt.plot(polydegree, estimated_mse_KFold, 'r--',  label = 'MSE KFold, k=5')
        plt.plot(polydegree, MSE_Bootstraps_compare, label='MSE Bootstrap, n=200, #bootstraps= 20')
        plt.xlabel('Pl. Degree')
        plt.ylabel('MSE')
        plt.legend()
        plt.show()
    elif k==10:
        plt.figure()
        plt.plot(polydegree, estimated_mse_sklearn, 'b', label = 'cross_val_score')
        plt.plot(polydegree, estimated_mse_KFold, 'r--',  label = 'MSE KFold, k=10')
        plt.plot(polydegree, MSE_Bootstraps_compare, label='MSE Bootstrap,n=200, #bootstraps= 20')
        plt.xlabel('Pl. Degree')
        plt.ylabel('MSE')
        plt.legend()
        plt.show()
    else:
        plt.figure()
        plt.plot(polydegree, estimated_mse_sklearn, 'b', label = 'cross_val_score')
        plt.plot(polydegree, estimated_mse_KFold, 'r--',  label = 'MSE LOO')
        plt.plot(polydegree, MSE_Bootstraps_compare, label='MSE Bootstrap,n=200, #bootstraps= 20')
        plt.xlabel('Pl. Degree')
        plt.ylabel('MSE')
        plt.legend()
        plt.show()